In [ ]:
import os

#!pip install langchain langchain-community langchain-experimental
#!pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFaceEndpoint, HuggingFacePipeline
from llama_cpp import Llama, LlamaGrammar
from langchain_community.llms.llamacpp import LlamaCpp

pipe = pipeline(
    "text-generation",
    model="mistralai/Mistral-Small-3.2-24B-Instruct-2506",
    device="cuda",
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)
local_llm = HuggingFacePipeline(
    pipeline=pipe
)

"""smart_llm = Llama(
    model_path="models/llama-3.2-3B-Instruct-Q4_0_4_4.gguf"
    #verbose=True
)"""

C:\Users\fbrossi\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fbrossi\.cache\huggingface\hub\models--mistralai--Mistral-Small-3.2-24B-Instruct-2506. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
from langchain_experimental.smart_llm import SmartLLMChain
from langchain_core.prompts import PromptTemplate

template = """
CASSETTE stands for "Cassette", "Cassetto", "AC", "RC"
then
CT stands for "Cash Transport", "Piatto", "Piatti", "LT", "ST", "PT"
then
NE stands for "Node Escrow", "precassa", "nastro"
then
NF stands for "Node Feeder", "bocchetta", "leva 1", "sfogliatore"
then
NV stands for "Node Validator", "leva 7", "validatore"
then
SHUTTER stands for "Shutter"

Your job is to classify the following input with the labels above defined.
You only answer with one single phrase of 50 words and the resulting classification, no more no less.
If you're unsure, return UNK.
After the answer write "end".

Input:
{input}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input"],
)

chain = prompt | local_llm

"""smart_chain = SmartLLMChain(
    llm=smart_llm,
    critique_llm=smart_llm,
    ideation_llm=smart_llm,
    prompt=prompt,
    n_ideas=2,
    verbose=True
)"""

'smart_chain = SmartLLMChain(\n    llm=smart_llm,\n    critique_llm=smart_llm,\n    ideation_llm=smart_llm,\n    prompt=prompt,\n    n_ideas=2,\n    verbose=True\n)'

In [55]:
def prompt_question(string):
    return chain.invoke({"input": string})

In [56]:
prompt_question("Riscontrata banconota inceppata nel CT. Rimozione e test ok")

TypeError: object of type 'StringPromptValue' has no len()